In [ ]:
import pandas as pd
import pickle
from collections import Counter
import glob

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
import numpy as np

In [ ]:
reduced=pickle.load(open('reduced_data_2.pkl',"rb"))

In [ ]:
X=reduced.drop(['length'],axis=1)
Y=reduced.length

n_X=(X-X.min())/(X.max()-X.min())

X_train, X_test, y_train, y_test = train_test_split(n_X, Y, test_size=0.25, random_state=42)

X_train=np.hstack((X_train,np.ones((X_train.shape[0],1))))
X_test=np.hstack((X_test,np.ones((X_test.shape[0],1))))

In [ ]:
import torch
from torch.autograd import Variable
import torch.nn.functional as F
import matplotlib.pyplot as plt

# torch can only train on Variable, so convert them to Variable
x, y = Variable(torch.from_numpy(X_train).float()), Variable(torch.from_numpy(y_train.values).float())

class Net(torch.nn.Module):
    def __init__(self, n_feature, n_hidden, n_output):
        super(Net, self).__init__()
        self.hidden = torch.nn.Linear(n_feature, n_hidden)   # hidden layer
        self.hidden2 = torch.nn.Linear(n_hidden, n_hidden)
        self.predict = torch.nn.Linear(n_hidden, n_output)   # output layer

    def forward(self, x):
        x = F.relu(self.hidden(x))
        x = F.relu(self.hidden2(x))      # activation function for hidden layer
        x = self.predict(x)             # linear output
        return x

net = Net(n_feature=254, n_hidden=64, n_output=1)     # define the network
print(net)  # net architecture

optimizer = torch.optim.Adam(net.parameters(), lr=0.5)
loss_func = torch.nn.MSELoss()  # this is for regression mean squared loss

plt.ion()   # something about plotting
for t in range(100):
    prediction = net(x)     # input x and predict based on x

    loss = loss_func(prediction, y)     # must be (1. nn output, 2. target)

    optimizer.zero_grad()   # clear gradients for next train
    loss.backward()         # backpropagation, compute gradients
    optimizer.step()        # apply gradients

    if t % 5 == 0:
        print('Loss=%.4f' % loss.data[0])

In [ ]:
prediction = net(Variable(torch.from_numpy(X_test).float()))   # input x and predict based on x
loss = loss_func(prediction, Variable(torch.from_numpy(y_test.values).float())) 

In [ ]:
loss.data[0]

In [ ]:
plt.scatter(y_test.values,net(Variable(torch.from_numpy(X_test).float())).data)
plt.show()

In [ ]:
X=reduced.drop(['length'],axis=1)
Y=reduced.length

def cat(v):
    if v>(-0.5) and v<0.5:
        return 0
    elif v>=0.5:
        return 1
    else:
        return -1

Y=Y.apply(cat)

n_X=(X-X.min())/(X.max()-X.min())

X_train, X_test, y_train, y_test = train_test_split(n_X, Y, test_size=0.25, random_state=42)

X_train=np.hstack((X_train,np.ones((X_train.shape[0],1))))
X_test=np.hstack((X_test,np.ones((X_test.shape[0],1))))

In [ ]:
from sklearn import ensemble

X = np.nan_to_num(X_train)
y = y_train.values

clf =ensemble.GradientBoostingClassifier()
clf.fit(X,y)

classifier = ensemble.RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 42)
classifier.fit(X, y)

y_pred = classifier.predict(np.nan_to_num(X_test))
reversefactor = dict(zip([-1,0,1],['Decrease','NotChanged','Increase']))
y_test = np.vectorize(reversefactor.get)(y_test)
y_pred = np.vectorize(reversefactor.get)(y_pred)
# Making the Confusion Matrix
print(pd.crosstab(y_test, y_pred, rownames=['Actual'], colnames=['Predicted']))